# Convolutional polynomial rings

**Definitions** 

* Ring of convolution polynomials  
$R = \frac{\mathbb{Z}[x]}{(x^N-1)}$

* Ring of convolution polynomials modulo q  
$R_q = \frac{(\mathbb{Z}/q\mathbb{Z})[x]}{(x^N-1)}$

* every elem of the ring has the form $0+a_1x+a_2x^2+···+a_{N−1}x^{N−1}$ with coeffs in its specific group

*Intuition*  
When we write mod $x^N-1$ we are simply requiring x^N to equal 1. So any time x^N appears, we replace it by 1.

*Notation:*  
We identify a polynomial with its coefficients $(a_0,a_1,a_2,...,a_{N−1})∈\mathbb{Z}^N$

*Addition*
$a(x)+b(x) = (a_0+b_0,a_1+b_1,a_2+b_2,...,a_{N−1}+b_{N−1})$

*Product* -> convolution  
$a(x) * b(x)=c(x)$ with $c_k= ∑_{i+j≡k(modN)}a_ib_{k−i}$

In [284]:
#example

In [285]:
N = 5
#create the polynomial integer ring
Zx.<x> = ZZ[]
#create the quotient ring -> not worth the hassle, we'll just use the convolution with 1
#R.<x> = F.quotient(y^N-1) 

#a = 1 - 2*x + 4*x**3 - x**4
a = Zx([1, -2, 0, 4, -1])
#b = 3 + 4*x - 2*x**2 + 5*x**3 + 2*x**4
b = Zx([3, 4, -2, 5, 2])
print(a)
print(b)

-x^4 + 4*x^3 - 2*x + 1
2*x^4 + 5*x^3 - 2*x^2 + 4*x + 3


In [286]:
c = a * b
print(c)

-2*x^8 + 3*x^7 + 22*x^6 - 16*x^5 + 5*x^4 + 21*x^3 - 10*x^2 - 2*x + 3


In [287]:
#Cool but we want the coefficients modulo x^n-1
c % (x^N-1)

5*x^4 + 19*x^3 - 7*x^2 + 20*x - 13

In [288]:
#lets reduce the coefficients to modulo 11
q = 11
(c % (x^N-1)) % 11

5*x^4 + 8*x^3 - 7*x^2 + 9*x + 9

In [289]:
#Lets define some functions that make this easy for us
def modulo(f, q):
    return Zx(list(coeff % q for coeff in f.list()))

def convolution(f, g, N, m = None):
    if m is None:
        return f*g % (x^N-1)
    else:
        return modulo(f*g % (x^N-1), m)

#the sage modulo leaks the sign of the coeffs so we defined our modulo
print((c % (x^N-1)) % 11)
print(modulo((c % (x^N-1)), 11))
#we use convolution with 1 to reduce in the ring R


5*x^4 + 8*x^3 - 7*x^2 + 9*x + 9
5*x^4 + 8*x^3 + 4*x^2 + 9*x + 9


In [290]:
c = convolution(a, b, N)
print(c)
c = convolution(a, b, N, q)
print(c)

5*x^4 + 19*x^3 - 7*x^2 + 20*x - 13
5*x^4 + 8*x^3 + 4*x^2 + 9*x + 9


**Definition**
Let $a(x)∈R_q$. The **center-lift** of $a(x)$ to $R$ is the unique polynomial $a′(x)∈R$ satisfying  
$a'(x)\mod q =a(x)$  
whose coefficients are chosen in the interval $\frac{−q}{2}<a'_i≤\frac{q}{2}$.

In [291]:
#Example
N = 5
q = 7
Zx.<x> = ZZ[]

In [292]:
a = 5 + 3*x - 6*x**2 + 2*x**3 + 4*x**4
print(a % 7)
print(modulo(a, 7))

4*x^4 + 2*x^3 - 6*x^2 + 3*x + 5
4*x^4 + 2*x^3 + x^2 + 3*x + 5


In [293]:
def center_lift(f, q):
    return Zx(list((coeff + q//2) % q - q//2 for coeff in f.list()))

In [294]:
a_lift = center_lift(modulo(a, q), q)
print(a_lift)

-3*x^4 + 2*x^3 + x^2 + 3*x - 2


In [295]:
modulo(a_lift, q) == modulo(a, q)

True

**Prop**  
Let $q$ be prime. Then $a(x)∈R_q$ has a multiplicative inverse <=> 
$\gcd(a(x),x^N−1)=1$ in $(\mathbb{Z}/q\mathbb{Z})[x]$.

If is true, then the inverse $a(x)^{−1}∈R_q$ can be computed using the extended Euclidean algorithm to find polynomials
$u(x),v(x)∈(\mathbb{Z}/q\mathbb{Z})[x]$ satisfying   
$a(x)u(x)+(x^N−1)v(x)=1$

Then $a(x)^{−1}=u(x)$ in $R_q$

In [296]:
#Example
N = 5
q = 2
Zx.<x> = ZZ[]

In [297]:
a = 1+x+x**4
a = convolution(a, Zx(1), N, q)
b = x^N-1
gcd(a, b)

1

In [298]:
_, a_inv, _ = xgcd(a, b)
convolution(a_inv, 1, N, q)

x^3 + x^2 + 1

In [299]:
convolution(a_inv * a, 1, N, q)

1

# NTRU 

## NTRU Encrypt

We need to define one more thing  
**Definition**

for any $d_1, d_2$ let $\mathcal{T}(d_1, d_2) = \Bigg\{ a(x) \in R \ \Bigg|\ 
\begin{align*}
&a(x) \text{ has } d_1 \text{ coeff } = 1 \\
&a(x) \text{ has } d_2 \text{ coeff } = -1 \\
&a(x) \text{ has the rest of coeff } = 0 \\
\end{align*} \Bigg\} $ = **Ternary polynomial**

In [300]:
#lets implement it
def randompoly_d(d1, d2, n):
    assert(d1 + d2 <=n)
    coeff_list = d1 * [1] + d2 * [-1] + (n-d1-d2) * [0]
    coeff_list = sample(coeff_list, len(coeff_list))
    return Zx(coeff_list)

In [301]:
randompoly_d(1, 2, 4)

-x^2 + x - 1

Let  
$R = \frac{(\mathbb{Z})[x]}{(x^N-1)}$
$R_q = \frac{(\mathbb{Z}/q\mathbb{Z})[x]}{(x^N-1)}$
$R_p = \frac{(\mathbb{Z}/p\mathbb{Z})[x]}{(x^N-1)}$



Public:
* Choose public parameters $(N,p,q,d)$ 
* $N, p$ = prime, $\gcd(p, q)=\gcd(N,q) = 1$
* $q>(6d+1)p$ <- **Important**

Key creation (Alice):
* Choose private $f \in \mathcal{T}(d+1, d) \text{ s.t } \ \exists \ f^{-1} \in R_q, R_p$
* Choose private $g \in \mathcal{T}(d, d)$
* $F_q = f^{-1}$ in $R_q$
* $F_p = f^{-1}$ in $R_p$
* return $h = F_q * g \in R_q$ 

Encryption (Bob):
* choose plaintext $m \in R_p$ with $m_i \in (-\frac p 2 , \frac p 2)$ =>  
=> the plaintext $m$ is a polynomial in $R$ that is the center lift of a polynomial in $R_p$
* choose $r \in \mathcal{T}(d, d)$
* return $e = p\cdot r*h+m \mod q$

Decryption (Alice):
* $a \equiv f * e \equiv p \cdot g*r+f*m \mod q$
* Center lift $a\in R$
* $m \equiv F_q * a \mod p$



Proof:  
The polynomials $g(x)$and $r(x)$ are in $\mathcal{T}(d, d)$, so if, in the convolution product $g(x)*r(x)$, all of their 1’s match up and all of their−1’s match up, the largest possible coefficient of $g(x)*r(x)$ is 2d.  
Similarly, $f(x)∈T(d+1,d)$ and the coefficients of $m(x)$ are between $-\frac p 2$ and $\frac p 2$, so the largest possible coefficient of $f(x)*m(x)$ is $(2d+1)\frac p 2$.  
So even if the largest coefficient of $g(x)*r(x)$ happens to coincide with the largest coefficient of $r(x)*m(x)$,the largest coefficient of $f*e$ has magnitude at most  
$p\cdot2d+(2d+1)\cdot \frac p 2 = (3d+12)\cdot p< \frac q 2$ (initial setting)  
Therefore $a = f * e \equiv p \cdot g*r+f*m$, is an equality in general, not just mod $q$.  
From here we go down mod $p$ and find $F_p(x) * a(x) \equiv m(x) \ \text{mod} \ p$

In [302]:
def invertmodprime(f,n,p):
    T = Zx.change_ring(Integers(p)).quotient(x^N-1)
    return Zx(lift(1 / T(f)))

def invertmodpowerof2(f, n, q):
    assert q.is_power_of(2)
    g = invertmodprime(f,2)
    while True:
        r = center_lift(convolution(g,f, n, q),q)
        if r == 1: 
            return g
        g = center_lift(convolution(g, 2 - r, n, q),q)

In [468]:
#Example
N, p, q, d = 7, 3, 41, 2

In [469]:
Zx.<x> = ZZ[]

In [303]:
def ntru_key(N, p, q, d):
    #get f, g
    f = randompoly_d(d+1, d, N)
    g = randompoly_d(d, d, N)
    #f = x**6-x**4+x**3+x**2-1
    #g = x**6+x**4-x**2-x
    
    #calculate inverses
    F_q = invertmodprime(f, N, q)
    F_p = invertmodprime(f, N, p)
    assert convolution(f, F_q, N, q) == 1
    assert convolution(f, F_p, N, p) == 1
    
     #calulate h
    h = convolution(F_q, g, N, q)
    return f, g, F_p, F_q, h

In [471]:
f, g, F_p, F_q, h = ntru_key(N, p, q, d)

In [472]:
print(f)
print(g)
print(F_p)
print(F_q)
print(h)

-x^5 + x^4 + x^3 + x^2 - 1
x^5 - x^4 + x^2 - x
x^4 + 2*x^3 + 2*x^2 + 2*x
38*x^6 + 6*x^5 + 30*x^4 + 23*x^3 + 36*x^2 + 10*x + 22
39*x^6 + 5*x^5 + 29*x^4 + 24*x^3 + 36*x^2 + 9*x + 22


In [473]:
m = Zx([1, -1, 1, 1, 0, -1])
m

-x^5 + x^3 + x^2 - x + 1

In [304]:
def ntru_encrypt(m, h, N, p, q, d):
    r = randompoly_d(d, d, N)
    #r = Zx([-1, 1, 0, 0, 0, -1, 1])
    e = modulo(convolution(p*r, h, N, q) + m, q)
    return e
    

In [475]:
e = ntru_encrypt(m, h, N, p, q, d)

In [476]:
e

20*x^6 + 35*x^5 + 36*x^4 + 26*x^3 + 39*x^2 + 31*x + 19

In [305]:
def ntru_decrypt(e, f, F_p, F_q, N, p, q, d):
    a = convolution(f, e, N, q)
    a_prime = center_lift(a, q)
    m = convolution(F_p, a_prime, N, p)
    return center_lift(m, p)

In [478]:
m_decr = ntru_decrypt(e, f, F_p, F_q, N, p, q, d)

In [479]:
m_decr, m_decr == m

(-x^5 + x^3 + x^2 - x + 1, True)

In [480]:
convolution(f, h, N, q)

x^5 + 40*x^4 + x^2 + 40*x

## NTRU as a lattice cryptosystem

**The NTRU Problem**
Given $h(x)$ find $f(x), g(x)$ s.t   
$f(x) * h(x) \equiv g(x) \ \text{mod} \ q$

**Remark**
There is not a unique solution. If $(g, f)$ is a solution then $(x^k g, x^kf)$ is a solution for all $k$

**Definitions**  
Let $h(x)=h_0+h_1x+···+h_{N−1}x^{N−1}$

Let $M^{NTRU}_h = 
\begin{bmatrix}
I_N & P_N(h) \\
0_n & qI_N
\end{bmatrix}
$
Where $P_N(h) = 
\begin{bmatrix}
h_0 & h_1 & ... & h_{N-1} \\
h_{N-1} & h_0 & ... & h_{N-2} \\
\vdots & \vdots & \ddots & \vdots \\
h_1 & h_2 & ... & h_0
\end{bmatrix}
$ is the permutation matrix

The lattice $L^{NTRU}_h = rowspan(M^{NTRU}_H)$

**OBS**: we will use column vectors so we transpose the stuff

**Prop**  
Let $f* h = g \% q; \ u \in R$ s.t. $f*h =g + qu$ Then: $[f, g] \in L^{NTRU}_h$ <=> $[f, -u] \cdot M^{NTRU}_h = [f, g]$

$\det (L^{NTRU}_h) = q^N$

*Modeling NTRU as SVP:*  
* Since $f, g$ are polynomials with coeff from $\{1, 0, -1\}$ => they have small norm =>  finding $(f, g)$ is like finding SVP in $L^{NTRU}_h$

*Modeling NTRU as CVP:*  
* $e = r*h + m \ \% \ q$ <=> $[0, e] = [0, r*h+m \ \% \ q] \equiv [r, r*h \ \% \ q] + [-r, m]$  
* We observe that $[-r,m]$ is short and $[0, r*h+m \ \% \ q] \in L^{NTRU}_h$
* Find $m$ from $h$ and $e$ as CVP to $[0, e]$

In [312]:
def generate_M_h(h, q, n, orientation = 'columns'):
    M = matrix(2*n)
    for i in range(n): M[i, i] = 1
    for i in range(n):
        M[i, n:] = [h.list()[n-i:] + h.list()[:n-i]] #permutations
    for i in range(n):
        M[i+n, i] = 0
    for i in range(n):
        M[i+n, i+n] = q
    
    if orientation == 'columns':
        return M.T
    else:
        return M
        
    

In [482]:
h

39*x^6 + 5*x^5 + 29*x^4 + 24*x^3 + 36*x^2 + 9*x + 22

In [483]:
M_h = generate_M_h(h, q, N, orientation = 'rows')

In [484]:
M_h

[ 1  0  0  0  0  0  0 22  9 36 24 29  5 39]
[ 0  1  0  0  0  0  0 39 22  9 36 24 29  5]
[ 0  0  1  0  0  0  0  5 39 22  9 36 24 29]
[ 0  0  0  1  0  0  0 29  5 39 22  9 36 24]
[ 0  0  0  0  1  0  0 24 29  5 39 22  9 36]
[ 0  0  0  0  0  1  0 36 24 29  5 39 22  9]
[ 0  0  0  0  0  0  1  9 36 24 29  5 39 22]
[ 0  0  0  0  0  0  0 41  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0 41  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0 41  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0 41  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 41  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0 41  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0 41]

# Lattice reduction Algorithms

## Gram schmidt

In [176]:
def gram_s(v):
    v = copy(v)
    n = v.nrows()
    m = v.ncols()
    v_s = matrix(QQ, n, m)
    mu = matrix(QQ, n, m)
    v_s[0] = v[0]
    for i in range(1, n):
        for j in range(i):
            mu[i, j] = float(v[i]*v_s[j] / norm(v_s[j])^2)
        v_s[i] = v[i] - sum([mu[i,j] * v_s[j] for j in range(0, i)])
        
    return v_s, mu
        

In [221]:
def calculate_mu(v, v_s):
    n = v.nrows()
    mu = matrix(QQ, v.nrows(), v.ncols())
    for i in range(n):
        for j in range(n):
            mu[i, j] = v[i] * v_s[j] / norm(v_s[j])**2
    return mu

In [217]:
v1=vector((1,3,2))
v2=vector((4,1,-2))
v3=vector((2,1,3))
v = matrix(QQ, (v1, v2, v3))

In [218]:
v_s, mu = gram_s(v)

In [219]:
v_s, mu

(
[     1      3      2]  [    0     0     0]
[ 53/14   5/14  -17/7]  [ 3/14     0     0]
[104/95 -26/19 143/95], [11/14  3/95     0]
)

In [222]:
mu2 = calculate_mu(v, v_s)

In [226]:
mu2

[    1     0     0]
[ 3/14     1     0]
[11/14  3/95     1]

In [180]:
v_s[1] * v_s[2]

0

In [181]:
v = matrix(QQ, ((4,1,3,-1),
                (2,1,-3,4),
                (1,0,-2,7)))

In [182]:
gram_s(v)

(
[       4        1        3       -1]
[   70/27    31/27    -23/9   104/27]
[-287/397 -405/397  799/397  844/397],

[      0       0       0       0]
[  -4/27       0       0       0]
[   -1/3 468/397       0       0]
)

## Gaussian Lattice reduction

* $L \subset \mathbb{R}^2, v_1, v_2 \in L, ||v_1|| < ||v_2||$ 
* project $v_2$ on the $v_1^\perp$ => $v_2^* = v_2 - \cfrac{v_1 \cdot v_2}{||v_1||^2} v_1$

But $\cfrac{v_1 \cdot v_2}{||v_1||^2} \notin \mathbb{Z} =>$ Let $m = \Big \lfloor \cfrac{v_1 \cdot v_2}{||v_1||^2} \Big \rceil$  
Then $v_2 = v_2 - mv_1$


In [550]:
vector([1, 1]) * vector([2, 2])

4

In [557]:
def gaussian_lattice_reduction(v1, v2):
    while True:
        if norm(v2) < norm(v1):
            v1, v2 = v2, v1
        m = round(v1 * v2 / norm(v1)**2)
        if m == 0:
            return v1, v2
        v2 = v2 - m*v1        

In [558]:
v1= vector((66586820,65354729))
v2= vector((6513996,6393464))

In [559]:
gaussian_lattice_reduction(v1,v2)

((2280, -1001), (-1324, -2376))

## LLL

Given a basis $\mathcal{B} = \{v_1, v_2, ... , v_n\}$ we want to find a "better" basis. What does "better" mean?
* short vectors
* as orthogonal as possible

Let $\mathcal{B}^* = \{v^∗_1,v^∗_2,...,v^∗_n\} = $ basis for span($\mathcal{B}$) constructed using Gram Schmidt

**Prop**  
$\det(L) = \det(F) = \det(F^*) = {\prod^n_{i=1}||v_i^*||}$

$\mathcal{B}$ is  LLL reduced <=>

* Size condition $|\mu_{i,j}| = \cfrac{|v_i \cdot v_i^*|} {||v_j^*||^2} \leq \cfrac 1 2 $ for all $1 \leq j < i \leq n$
* Lovasz Condition $||v^*_i||^2 \geq \left(\cfrac 3 4 - \mu^2_{i, i-1} \right)||v_{i-1}^*||^2$ for all $1 < i \leq n$ <=> $||Proj_{span(v_1,...v_{i-2})^\perp}(v_i)|| \geq \frac 3 4 ||Proj_{span(v_1,...v_{i-2})^\perp} (v_{i-1})||$ 

intuition for lovasz condition: https://crypto.stackexchange.com/questions/39532/why-is-the-lov%C3%A1sz-condition-used-in-the-lll-algorithm

**Theorem**
Let $L$ = lattice, $\dim(L) = n$. Any LLL reduced basis $\{v_1,v_2,...,v_n\}$ for $L$ has the following two properties:
* $\prod^n_{i=1}||v_i|| \leq 2^{n(n-1)/4} \cdot \det L$
* $||v_j|| \leq 2^{(i-1)/2}||v_i^*||$ for all $1 \leq j \leq i \leq n$

Further
* $||v_1|| \leq 2^{(n-1)/4} |\det L|^{1/n}$
* $||v_1|| \leq 2^{(n-1)/2} \underset{0 \neq v \in L}{\min}||v||$ => LLL sovles apprSVP within $2^{(n-1)/2}$

LLL algorithm intuition: https://kel.bz/post/lll/

algorithm implemented after this: https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm

*Algorithm details*  

* The goal of LLL is to produce a list of short vectors in increasing order oflength. For each $1≤l≤n$,let $L_l$ denote the lattice spanned by $v_1,...,v_l$.  
* What LLL attempts to do is to find an ordering of the basis vectors (combined with size reductions whenever possible) that minimizes the determinants $\det(L_l)$   
* If the number 3/4  is replaced by the number 1, then the LLL algorithm does precisely this; it swaps $v_k$ and $v_{k−1} whenever doing so reduces the value of det $L_{k−1}$. Unfortunately, if we use 1 instead of 3/4, then it is an open problem whether the LLL algorithm terminates in polynomial time

In [318]:
def lenstra_lenstra_lovasz(v):    
    v = copy(v)
    n = v.nrows()
    v_s, mu= gram_s(v)
    k = 1
    while k < n:
        for j in range(k-1, -1, -1):
            if abs(mu[k, j]) > 1/2:
                v[k] = v[k] - round(mu[k, j]) * v[j]
                v_s, mu= gram_s(v)
        #Lovasz condition
        if norm(v_s[k])^2 > (3/4 - round(mu[k, k-1])^2) * norm(v_s[k-1])^2 :
            k = k+1
        else:
            v[k-1], v[k] = v[k], v[k-1]
            v_s, mu= gram_s(v)
            k = max(k-1, 1)
    return v

In [280]:
M = matrix([
    [19, 2,32, 46, 3, 33],
    [15, 42, 11,  0,  3, 24],
    [43, 15,  0, 24,  4, 16], 
    [20, 44, 44,  0, 18, 15], 
    [0, 48 ,35, 16, 31, 31], 
    [48, 33, 32, 9, 1, 29]
])

In [281]:
M_lll = lenstra_lenstra_lovasz(M)

[15 42 11  0  3 24]
[19  2 32 46  3 33]
[43 15  0 24  4 16]
[20 44 44  0 18 15]
[ 0 48 35 16 31 31]
[48 33 32  9  1 29]
[ 15  42  11   0   3  24]
[ 28 -27 -11  24   1  -8]
[  4 -40  21  46   0   9]
[ 20  44  44   0  18  15]
[  0  48  35  16  31  31]
[ 48  33  32   9   1  29]
[ 28 -27 -11  24   1  -8]
[ 15  42  11   0   3  24]
[  4 -40  21  46   0   9]
[ 20  44  44   0  18  15]
[  0  48  35  16  31  31]
[ 48  33  32   9   1  29]
[ 28 -27 -11  24   1  -8]
[ 15  42  11   0   3  24]
[  5   2  33   0  15  -9]
[-24 -13  32  22  -1  17]
[  0  48  35  16  31  31]
[ 48  33  32   9   1  29]
[ 28 -27 -11  24   1  -8]
[  5   2  33   0  15  -9]
[ 15  42  11   0   3  24]
[-24 -13  32  22  -1  17]
[  0  48  35  16  31  31]
[ 48  33  32   9   1  29]
[  5   2  33   0  15  -9]
[ 28 -27 -11  24   1  -8]
[ 15  42  11   0   3  24]
[-24 -13  32  22  -1  17]
[  0  48  35  16  31  31]
[ 48  33  32   9   1  29]
[  5   2  33   0  15  -9]
[ 28 -27 -11  24   1  -8]
[ 15  42  11   0   3  24]
[-20   4  -9  16  13  

In [245]:
M_lll

[ -7  12   8  -4 -19  -9]
[-20   4  -9  16  13  16]
[  6   7  20  21  -8  12]
[-28  11  12  -9  17 -14]
[-10 -24  21 -15  -6 -11]
[ -7  -4   9  11  -1 -31]

In [246]:
M.LLL()

[ -7  12   8  -4 -19  -9]
[-20   4  -9  16  13  16]
[  6   7  20  21  -8  12]
[-28  11  12  -9  17 -14]
[ -7  -4   9  11  -1 -31]
[-10 -24  21 -15  -6 -11]

## More algorithms

* Babai's nearest plane https://cims.nyu.edu/~regev/teaching/lattices_fall_2004/ln/cvp.pdf
* KZ-LLL and BKZ-LLL  https://arxiv.org/pdf/0801.3331.pdf

# Breaking stuff

# GO TO THE PYTHON NB FOR THE OTHER ALGORITHMS

## NTRU

In [306]:
#Example
N, p, q, d = 7, 3, 41, 2
Zx.<x> = ZZ[]

In [307]:
f, g, F_p, F_q, h = ntru_key(N, p, q, d)
print(f)
print(g)
print(h)

-x^6 - x^5 + x^3 + x + 1
x^6 + x^4 - x^3 - x^2
3*x^6 + 7*x^5 + 25*x^4 + 11*x^3 + 16*x^2 + 16*x + 4


In [338]:
m = Zx([1, -1, 1, 1, 0, -1])

e = ntru_encrypt(m, h, N, p, q, d)
m_decr = ntru_decrypt(e, f, F_p, F_q, N, p, q, d)
m_decr == m

True

In [319]:
Mh = generate_M_h(h, q, N, orientation = 'rows')

In [320]:
print(Mh)

[ 1  0  0  0  0  0  0  4 16 16 11 25  7  3]
[ 0  1  0  0  0  0  0  3  4 16 16 11 25  7]
[ 0  0  1  0  0  0  0  7  3  4 16 16 11 25]
[ 0  0  0  1  0  0  0 25  7  3  4 16 16 11]
[ 0  0  0  0  1  0  0 11 25  7  3  4 16 16]
[ 0  0  0  0  0  1  0 16 11 25  7  3  4 16]
[ 0  0  0  0  0  0  1 16 16 11 25  7  3  4]
[ 0  0  0  0  0  0  0 41  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0 41  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0 41  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0 41  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0 41  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0 41  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0 41]


In [321]:
Mh_lll = lenstra_lenstra_lovasz(Mh)

In [322]:
Mh_lll

[ 0  1  1 -1 -1  0 -1 -1  0 -1  0  0  1  1]
[-1  0  1  1 -1 -1  0  1 -1  0 -1  0  0  1]
[ 0 -1  0  1  1 -1 -1  1  1 -1  0 -1  0  0]
[-1  0 -1  0  1  1 -1  0  1  1 -1  0 -1  0]
[-1 -1  0 -1  0  1  1  0  0  1  1 -1  0 -1]
[ 0 -2  0  1 -1  1  1 -1  1  0  1  0 -2  1]
[ 1  1 -1 -1  0 -1  0  0 -1  0  0  1  1 -1]
[-2  1 -2  1  9 -6 -1 -7 -7  4  8 -3 -1  6]
[ 1 -2 -8  7  1  3 -1 -4 -7  3  1 -7  6  8]
[-7  9  0  5 -2 -8  4 -3  8  0  2  0 -1 -6]
[-5  6 -8  1 -5  3  8  6  4 -8  0 -3 -1  2]
[-4  3 -6  4  3 -2  2  2  2  6 10  3  9  9]
[ 9  0  5 -2 -8  4 -7  8  0  2  0 -1 -6 -3]
[ 8 -4  7 -9  0 -5  2  1  6  3 -8  0 -2  0]

In [323]:
import numpy as np
def hadamards_ratio(v):
    '''vectors are columns'''
    H = (np.abs(np.linalg.det(v)) / np.prod([np.linalg.norm(v_i) for v_i in v.T])) ** (1/v.shape[1])
    return H

In [326]:
hadamards_ratio(np.array(Mh).T), hadamards_ratio(np.array(Mh_lll).T)

(0.16549789057036002, 0.8311505086376579)

In [329]:
v0 = Mh_lll[0]
v0, len(v0)

((0, 1, 1, -1, -1, 0, -1, -1, 0, -1, 0, 0, 1, 1), 14)

In [332]:
f_eve = Zx(list(v0[:7]))
g_eve = Zx(list(v0[7:]))
print(f_eve)
print(g_eve)
#these keys are just rotations of the original key, so they are solution to decrypt

-x^6 - x^4 - x^3 + x^2 + x
x^6 + x^5 - x^2 - 1


In [340]:
F_p_eve = invertmodprime(f_eve, N, p)
print(F_p_eve)

2*x^6 + x^3 + 2*x^2


In [341]:
m_decr_eve = ntru_decrypt(e, f_eve, F_p_eve, None, N, p, q, None)
print(m_decr_eve)
print(m_decr_eve == m)

-x^5 + x^3 + x^2 - x + 1
True
